In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
import requests as rq
import lxml 
import time
import pandas as pd
import numpy as np
import random
import pyprind

# Step 1: Get the URL of the webpage of every district in Tokyo.
Including the 23 special wards(特別区), cities(市), towns(町) , and villages(村).


Set up the ChromeDriver.

In [7]:
browser = webdriver.Chrome("C:/Users/Nolan/Desktop/.vscode/chromedriver")
browser.get('https://suumo.jp/chintai/tokyo/city/')
browser.maximize_window()

Get URLs

%%html
<figure class="half">
    <img src="checkbox.png" width="40%">  <img src="search button.png" width="40%">
</figure>

In [8]:
boxes = browser.find_elements_by_name('sc') #The list of all checkboxes
District_URLs = pd.DataFrame(columns=['ID', 'District', 'URL']) #Create a dataframe to store all the URLs
for i in range(0,len(boxes)):
    browser.get('https://suumo.jp/chintai/tokyo/city/') 
    boxes = browser.find_elements_by_name('sc') # Find all the checkboxes
    label = browser.find_elements_by_tag_name('label') #The name tag of each checkbox

    boxes[i].click() #Click the checkbox
    idd = boxes[i].get_attribute('id') #Record the ID of the checkboxes
    labelid = label[i].get_attribute('for') #Record the ID of the label

    if (labelid == idd): 
        district = label[i].text
        district = district.split('(')[0] #Make sure that each checkboxes have a corresponding name tag 
    else:
        pass

    Search_button = browser.find_element_by_xpath('//*[@id="js-searchpanel"]/div/div/a') #The search button
    Search_button.click()
    url = browser.current_url #Get the url of current page
    District_URLs = District_URLs.append({'ID' : idd, 'District': district, 'URL' : url}, ignore_index=True)
    
    time.sleep(random.randint(0,2))
    
District_URLs.to_csv("E:/UTokyo/Prof_Griffen RA/District_URLs(0609).csv") 

## Step 2: Get all the information of the porperty

### Details of the property

In [9]:
def Additional_Info(birubody):

    detail = birubody.find("td", class_="ui-text--midium ui-text--bold")
    detail_a = detail.a['href']
    detail_url = 'http://suumo.jp'+ detail_a
    detail_response = rq.get(detail_url)
    detail_soup = BeautifulSoup(detail_response.text, "lxml")

    features = detail_soup.find("div", class_ = "bgc-wht ol-g") #Building Feature

    title = detail_soup.find("h1", class_="section_h1-header-title") #Building Title
    if(title is None):
        title = "None"
    else:
        title = title.text
    #Information in the property description table
    gaiyou_table = detail_soup.find("table", class_ ="data_table table_gaiyou") #Gaiyou_Table

    if(gaiyou_table is None):
        layout_detail = "None" #[0,0] Layout detail 
        building_material = "None" #[0,1] Building material 
        build_date = "None" #[1,1] Date of build 
        nonlife_insurance = "None" #[2,0] Non-life insurance 
        parkinglot = "None" #[2,1] Parking lot 
        move_in_date = "None" #[3,0] Date can move in  
        trans_mode = "None" #[3,1] Transaction mode 
        requirements = "None" #[4,0] Other requirements 
        HS_propertycode = "None" #[4,1] Handling store property code
        SUUMO_code = "None" #[5,0] SUUMO object code
        n_of_HH = "None" #[5,1] number of households

    else:
        layout_detail = gaiyou_table.find_all("tr")[0].find_all("td")[0].text.strip() #[0,0] Layout detail 
        building_material = gaiyou_table.find_all("tr")[0].find_all("td")[1].text.strip() #[0,1] Building material 
        build_date = gaiyou_table.find_all("tr")[1].find_all("td")[1].text.strip() #[1,1] Date of build 
        nonlife_insurance = gaiyou_table.find_all("tr")[2].find_all("td")[0].text.strip() #[2,0] Non-life insurance 
        parkinglot = gaiyou_table.find_all("tr")[2].find_all("td")[1].text.strip() #[2,1] Parking lot 
        move_in_date = gaiyou_table.find_all("tr")[3].find_all("td")[0].text.strip() #[3,0] Date can move in  
        trans_mode = gaiyou_table.find_all("tr")[3].find_all("td")[1].text.strip() #[3,1] Transaction mode 
        requirements = gaiyou_table.find_all("tr")[4].find_all("td")[0].text.strip() #[4,0] Other requirements 
        HS_propertycode = gaiyou_table.find_all("tr")[4].find_all("td")[1].text.strip() #[4,1] Handling store property code
        SUUMO_code = gaiyou_table.find_all("tr")[5].find_all("td")[0].text #[5,0] SUUMO object code
        n_of_HH = gaiyou_table.find_all("tr")[5].find_all("td")[1].text.strip() #[5,1] number of households
    
    #Update Note
    Update_note = detail_soup.find("div", class_ ="captiontext l-space_medium") #Update Date

    if(Update_note is None):
        Update_note = "Omitted"
    else:
        Update_note = Update_note.text.strip()


    if (features is None):
        return("No longer availiable", title, layout_detail, building_material, build_date, nonlife_insurance, parkinglot, move_in_date, trans_mode, requirements, HS_propertycode, SUUMO_code, n_of_HH, Update_note)
    else:
        features_text = features.get_text().strip()
        time.sleep(random.uniform(0.5,0.6))
        return(features_text, title, layout_detail, building_material, build_date, nonlife_insurance, parkinglot, move_in_date, trans_mode, requirements, HS_propertycode, SUUMO_code, n_of_HH, Update_note)

In [10]:
def getInfo(District_url):
    d = []
    global df
    #s = 0
    response = rq.get(District_url) 
    soup = BeautifulSoup(response.text, "lxml")
    biru_set = soup.find_all("div", "cassetteitem")

    for i in range(len(biru_set)):
        biru_body = biru_set[i].find_all("tbody")
        title = biru_set[i].find("div","cassetteitem_content-title") #Name of the building
        location = biru_set[i].find("li", "cassetteitem_detail-col1") #Location of the building
        district = location.text.split(sep = "区")[0]
        district = district + "区" #Split the location and extract "district" in the address
        age = biru_set[i].find("li", "cassetteitem_detail-col3").find_all("div")[0] #Age of the building
        totalfloor = biru_set[i].find("li", "cassetteitem_detail-col3").find_all("div")[1] #Total floor count of the building
        for j in range(len(biru_body)):
            floor = biru_body[j].find_all("td")[2]
            floortext = floor.text.replace('											', '') #Floor
            deposit = biru_body[j].find("span", class_ = "cassetteitem_price cassetteitem_price--deposit") #Deposit
            gratuity = biru_body[j].find("span", class_ = "cassetteitem_price cassetteitem_price--gratuity") #Gratuity
            administration = biru_body[j].find("span", class_ = "cassetteitem_price cassetteitem_price--administration")
            rent = biru_body[j].find("span", class_ = "cassetteitem_other-emphasis ui-text--bold") #Rent
            madori = biru_body[j].find("span", class_ = "cassetteitem_madori") #Layout
            menseki = biru_body[j].find("span", class_ = "cassetteitem_menseki") #Area
            more = Additional_Info(biru_body[j]) #More detailed information

            d.append((i+1, title.text, district, location.text, floortext.strip(), totalfloor.text, deposit.text, gratuity.text, administration.text, rent.text, madori.text, menseki.text, age.text, more[0], more[1], more[2], more[3], more[4], more[5], more[6], more[7], more[8], more[9], more[10], more[11], more[12], more[13]))

        df = pd.DataFrame(d, columns=['BuildingCount', 'BuildingName', 'District', 'Location', 'Floor', 'TotalFloor', 'Deposit敷', 'Gratuity礼', 'AdministrationFee', 'Rent', 'Madori', 'Menseki', 'Age', 'Details', 'Room_Name', 'Layout_detail', 'Building_material', 'Build_date', 'Nonlife_insurance', 'Parking_lot', 'Move_in_date', 'Transaction_mode', 'Requirements', 'HS_propertycode', 'SUUMO object code', 'Number_of_households', 'Update_note'])


    return(df)

#### For testing getInfo()

In [ ]:
District_URLs = pd.read_csv("E:/UTokyo/Prof_Griffen RA/District_URLs(0408).csv")
District_url = District_URLs.loc[4,"URL"]
a = getInfo(District_url)
a.loc[4]

In [11]:
def TurnPage(District_url):
    global df_everypage 
    df_everypage =  pd.DataFrame()
    response = rq.get(District_url) 
    soup = BeautifulSoup(response.text, "lxml")
    pagesection = soup.find("div", class_ = "pagination pagination_set-nav") # Find the page button section
    totalpagescount = pagesection.find("ol", "pagination-parts").find_all("li")[-1].text # The number on the last page button is the total number of the pages that we have to page through
    
    for p in range(1,int(totalpagescount)+1):#
        urls = District_url +'&et=9999999&page1&page='+str(p)
        getInfo(urls)
        df["From Page #"] = p
        df_everypage = df_everypage.append(df)
        time.sleep(random.uniform(1,2))
    return(df_everypage)


## Execute

In [12]:
District_URLs = pd.read_csv("E:/UTokyo/Prof_Griffen RA/District_URLs(0609).csv")
len(District_URLs)

50

In [13]:
for i in range(0,3):
    print(District_URLs.loc[i,'District'])
    print(District_URLs.loc[i,'URL'])

千代田区
https://suumo.jp/jj/chintai/ichiran/FR301FC001/?ar=030&bs=040&ta=13&sc=13101&cb=0.0&ct=9999999&et=9999999&cn=9999999&mb=0&mt=9999999&shkr1=03&shkr2=03&shkr3=03&shkr4=03&fw2=&srch_navi=1
中央区
https://suumo.jp/jj/chintai/ichiran/FR301FC001/?ar=030&bs=040&ta=13&sc=13102&cb=0.0&ct=9999999&et=9999999&cn=9999999&mb=0&mt=9999999&shkr1=03&shkr2=03&shkr3=03&shkr4=03&fw2=&srch_navi=1
港区
https://suumo.jp/jj/chintai/ichiran/FR301FC001/?ar=030&bs=040&ta=13&sc=13103&cb=0.0&ct=9999999&et=9999999&cn=9999999&mb=0&mt=9999999&shkr1=03&shkr2=03&shkr3=03&shkr4=03&fw2=&srch_navi=1


In [7]:
for i in range(0,1):
    print(i)

0


In [15]:
len(District_URLs[0:1])

1

In [ ]:
District_URLs[0:1]

都心部

In [14]:
Tokyo_Dataset = pd.DataFrame()
pbar = pyprind.ProgBar(len(District_URLs[0:1]), monitor=True)
for i in range(0,1):
    D = TurnPage(District_URLs.loc[i,'URL'])
    print(District_URLs.loc[i,'District'])
    Tokyo_Dataset = Tokyo_Dataset.append(D)
    pbar.update()
    print(pbar)

0% [#] 100% | ETA: 00:00:00
Total time elapsed: 00:37:22
千代田区
Title: 
  Started: 06/09/2020 15:18:11
  Finished: 06/09/2020 15:55:34
  Total time elapsed: 00:37:22
  CPU %: 13.00
  Memory %: 2.18


In [16]:
save_path = "E:/UTokyo/Prof_Griffen RA/June_SUUMO/"+str(District_URLs.loc[0,'District']).strip()+"(0609).csv"

In [17]:
Tokyo_Dataset.to_csv(save_path, encoding = "'utf_8_sig'")

In [19]:
Tokyo_Dataset = pd.DataFrame()
pbar = pyprind.ProgBar(len(District_URLs[1:2]), monitor=True)
for i in range(1,2):
    D = TurnPage(District_URLs.loc[i,'URL'])
    print(District_URLs.loc[i,'District'])
    Tokyo_Dataset = Tokyo_Dataset.append(D)
    pbar.update()
    print(pbar)

    save_path = "E:/UTokyo/Prof_Griffen RA/June_SUUMO/"+str(District_URLs.loc[i,'District']).strip()+"(0509).csv"
    Tokyo_Dataset.to_csv(save_path, encoding = "'utf_8_sig'")
    Tokyo_Dataset = pd.DataFrame()
    time.sleep(10)

0% [# ] 100% | ETA: 01:15:12中央区
Title: 
  Started: 06/09/2020 16:21:14
  Finished: 06/09/2020 17:36:27
  Total time elapsed: 00:00:00
  CPU %: 12.80
  Memory %: 2.31


ConnectionError: ('Connection aborted.', OSError("(10051, 'WSAENETUNREACH')"))

In [20]:
Tokyo_Dataset = pd.DataFrame()
pbar = pyprind.ProgBar(len(District_URLs[2:3]), monitor=True)
for i in range(2,3):
    D = TurnPage(District_URLs.loc[i,'URL'])
    print(District_URLs.loc[i,'District'])
    Tokyo_Dataset = Tokyo_Dataset.append(D)
    pbar.update()
    print(pbar)

    save_path = "E:/UTokyo/Prof_Griffen RA/June_SUUMO/"+str(District_URLs.loc[i,'District']).strip()+"(0509).csv"
    Tokyo_Dataset.to_csv(save_path, encoding = "'utf_8_sig'")
    Tokyo_Dataset = pd.DataFrame()
    time.sleep(10)

0% [#] 100% | ETA: 00:00:00
Total time elapsed: 02:09:23
港区
Title: 
  Started: 06/09/2020 20:09:03
  Finished: 06/09/2020 22:18:27
  Total time elapsed: 02:09:23
  CPU %: 12.80
  Memory %: 2.78


In [21]:
Tokyo_Dataset = pd.DataFrame()
pbar = pyprind.ProgBar(len(District_URLs[3:6]), monitor=True)
for i in range(3,6):
    D = TurnPage(District_URLs.loc[i,'URL'])
    print(District_URLs.loc[i,'District'])
    Tokyo_Dataset = Tokyo_Dataset.append(D)
    pbar.update()
    print(pbar)

    save_path = "E:/UTokyo/Prof_Griffen RA/June_SUUMO/"+str(District_URLs.loc[i,'District']).strip()+"(0609).csv"
    Tokyo_Dataset.to_csv(save_path, encoding = "'utf_8_sig'")
    Tokyo_Dataset = pd.DataFrame()
    time.sleep(10)

新宿区
Title: 
  Started: 06/09/2020 23:51:11
  Finished: 06/10/2020 02:46:10
  Total time elapsed: 00:00:00
  CPU %: 12.80
  Memory %: 2.90
0% [## ] 100% | ETA: 02:06:45文京区
Title: 
  Started: 06/09/2020 23:51:11
  Finished: 06/10/2020 04:04:41
  Total time elapsed: 00:00:00
  CPU %: 12.20
  Memory %: 2.82
0% [###] 100% | ETA: 00:00:00
Total time elapsed: 06:18:19
渋谷区
Title: 
  Started: 06/09/2020 23:51:11
  Finished: 06/10/2020 06:09:30
  Total time elapsed: 06:18:19
  CPU %: 12.30
  Memory %: 2.85


23区東部

In [22]:
Tokyo_Dataset = pd.DataFrame()
pbar = pyprind.ProgBar(len(District_URLs[6:9]), monitor=True)
for i in range(6,9):
    D = TurnPage(District_URLs.loc[i,'URL'])
    print(District_URLs.loc[i,'District'])
    Tokyo_Dataset = Tokyo_Dataset.append(D)
    pbar.update()
    print(pbar)

    save_path = "E:/UTokyo/Prof_Griffen RA/June_SUUMO/"+str(District_URLs.loc[i,'District']).strip()+"(0610).csv"
    Tokyo_Dataset.to_csv(save_path, encoding = "'utf_8_sig'")
    Tokyo_Dataset = pd.DataFrame()
    time.sleep(10)

台東区
Title: 
  Started: 06/10/2020 09:45:33
  Finished: 06/10/2020 11:24:06
  Total time elapsed: 00:00:00
  CPU %: 12.10
  Memory %: 2.65
0% [## ] 100% | ETA: 01:32:21墨田区
Title: 
  Started: 06/10/2020 09:45:33
  Finished: 06/10/2020 12:50:17
  Total time elapsed: 00:00:00
  CPU %: 11.90
  Memory %: 2.46
0% [###] 100% | ETA: 00:00:00
Total time elapsed: 05:10:19
江東区
Title: 
  Started: 06/10/2020 09:45:33
  Finished: 06/10/2020 14:55:53
  Total time elapsed: 05:10:19
  CPU %: 12.40
  Memory %: 2.67


In [23]:
Tokyo_Dataset = pd.DataFrame()
pbar = pyprind.ProgBar(len(District_URLs[9:11]), monitor=True)
for i in range(9,11):
    D = TurnPage(District_URLs.loc[i,'URL'])
    print(District_URLs.loc[i,'District'])
    Tokyo_Dataset = Tokyo_Dataset.append(D)
    pbar.update()
    print(pbar)

    save_path = "E:/UTokyo/Prof_Griffen RA/June_SUUMO/"+str(District_URLs.loc[i,'District']).strip()+"(0610).csv"
    Tokyo_Dataset.to_csv(save_path, encoding = "'utf_8_sig'")
    Tokyo_Dataset = pd.DataFrame()
    time.sleep(10)

0% [# ] 100% | ETA: 01:00:19荒川区
Title: 
  Started: 06/10/2020 15:28:03
  Finished: 06/10/2020 16:28:23
  Total time elapsed: 00:00:00
  CPU %: 28.10
  Memory %: 2.46
0% [##] 100% | ETA: 00:00:00
Total time elapsed: 03:14:41
足立区
Title: 
  Started: 06/10/2020 15:28:03
  Finished: 06/10/2020 18:42:45
  Total time elapsed: 03:14:41
  CPU %: 14.00
  Memory %: 2.43


In [24]:
Tokyo_Dataset = pd.DataFrame()
pbar = pyprind.ProgBar(len(District_URLs[11:13]), monitor=True)
for i in range(11,13):
    D = TurnPage(District_URLs.loc[i,'URL'])
    print(District_URLs.loc[i,'District'])
    Tokyo_Dataset = Tokyo_Dataset.append(D)
    pbar.update()
    print(pbar)

    save_path = "E:/UTokyo/Prof_Griffen RA/June_SUUMO/"+str(District_URLs.loc[i,'District']).strip()+"(0610).csv"
    Tokyo_Dataset.to_csv(save_path, encoding = "'utf_8_sig'")
    Tokyo_Dataset = pd.DataFrame()
    time.sleep(10)

0% [# ] 100% | ETA: 01:24:33葛飾区
Title: 
  Started: 06/10/2020 21:53:44
  Finished: 06/10/2020 23:18:17
  Total time elapsed: 00:00:00
  CPU %: 12.90
  Memory %: 2.47
0% [##] 100% | ETA: 00:00:00
Total time elapsed: 03:37:09
江戸川区
Title: 
  Started: 06/10/2020 21:53:44
  Finished: 06/11/2020 01:30:53
  Total time elapsed: 03:37:09
  CPU %: 14.20
  Memory %: 2.52


23区南部

In [25]:
Tokyo_Dataset = pd.DataFrame()
pbar = pyprind.ProgBar(len(District_URLs[13:17]), monitor=True)
for i in range(13,17):
    D = TurnPage(District_URLs.loc[i,'URL'])
    print(District_URLs.loc[i,'District'])
    Tokyo_Dataset = Tokyo_Dataset.append(D)
    pbar.update()
    print(pbar)

    save_path = "E:/UTokyo/Prof_Griffen RA/June_SUUMO/"+str(District_URLs.loc[i,'District']).strip()+"(0611).csv"
    Tokyo_Dataset.to_csv(save_path, encoding = "'utf_8_sig'")
    Tokyo_Dataset = pd.DataFrame()
    time.sleep(10)

0% [#   ] 100% | ETA: 06:32:10品川区
Title: 
  Started: 06/11/2020 01:53:36
  Finished: 06/11/2020 04:04:19
  Total time elapsed: 00:00:00
  CPU %: 12.00
  Memory %: 2.74
0% [##  ] 100% | ETA: 03:38:14目黒区
Title: 
  Started: 06/11/2020 01:53:36
  Finished: 06/11/2020 05:31:50
  Total time elapsed: 00:00:00
  CPU %: 11.70
  Memory %: 2.72


ConnectionError: ('Connection aborted.', OSError("(10054, 'WSAECONNRESET')"))

In [27]:
for i in range(15,17):
    print(District_URLs.loc[i,'District'])
    print(District_URLs.loc[i,'URL'])

大田区
https://suumo.jp/jj/chintai/ichiran/FR301FC001/?ar=030&bs=040&ta=13&sc=13111&cb=0.0&ct=9999999&et=9999999&cn=9999999&mb=0&mt=9999999&shkr1=03&shkr2=03&shkr3=03&shkr4=03&fw2=&srch_navi=1
世田谷区
https://suumo.jp/jj/chintai/ichiran/FR301FC001/?ar=030&bs=040&ta=13&sc=13112&cb=0.0&ct=9999999&et=9999999&cn=9999999&mb=0&mt=9999999&shkr1=03&shkr2=03&shkr3=03&shkr4=03&fw2=&srch_navi=1


In [28]:
Tokyo_Dataset = pd.DataFrame()
pbar = pyprind.ProgBar(len(District_URLs[15:17]), monitor=True)
for i in range(15,17):
    D = TurnPage(District_URLs.loc[i,'URL'])
    print(District_URLs.loc[i,'District'])
    Tokyo_Dataset = Tokyo_Dataset.append(D)
    pbar.update()
    print(pbar)

    save_path = "E:/UTokyo/Prof_Griffen RA/June_SUUMO/"+str(District_URLs.loc[i,'District']).strip()+"(0611).csv"
    Tokyo_Dataset.to_csv(save_path, encoding = "'utf_8_sig'")
    Tokyo_Dataset = pd.DataFrame()
    time.sleep(10)

0% [# ] 100% | ETA: 02:50:54大田区
Title: 
  Started: 06/11/2020 11:03:32
  Finished: 06/11/2020 13:54:27
  Total time elapsed: 00:00:00
  CPU %: 12.80
  Memory %: 2.40
0% [##] 100% | ETA: 00:00:00
Total time elapsed: 07:00:34
世田谷区
Title: 
  Started: 06/11/2020 11:03:32
  Finished: 06/11/2020 18:04:06
  Total time elapsed: 07:00:34
  CPU %: 13.20
  Memory %: 2.67


23区西部

In [29]:
Tokyo_Dataset = pd.DataFrame()
pbar = pyprind.ProgBar(len(District_URLs[17:19]), monitor=True)
for i in range(17,19):
    D = TurnPage(District_URLs.loc[i,'URL'])
    print(District_URLs.loc[i,'District'])
    Tokyo_Dataset = Tokyo_Dataset.append(D)
    pbar.update()
    print(pbar)

    save_path = "E:/UTokyo/Prof_Griffen RA/June_SUUMO/"+str(District_URLs.loc[i,'District']).strip()+"(0509).csv"
    Tokyo_Dataset.to_csv(save_path, encoding = "'utf_8_sig'")
    Tokyo_Dataset = pd.DataFrame()
    time.sleep(10)

0% [# ] 100% | ETA: 02:00:09中野区
Title: 
  Started: 06/11/2020 18:04:07
  Finished: 06/11/2020 20:04:16
  Total time elapsed: 00:00:00
  CPU %: 13.40
  Memory %: 2.52
0% [##] 100% | ETA: 00:00:00
Total time elapsed: 05:04:04
杉並区
Title: 
  Started: 06/11/2020 18:04:07
  Finished: 06/11/2020 23:08:11
  Total time elapsed: 05:04:04
  CPU %: 13.60
  Memory %: 2.42


In [30]:
Tokyo_Dataset = pd.DataFrame()
pbar = pyprind.ProgBar(len(District_URLs[19:20]), monitor=True)
for i in range(19,20):
    D = TurnPage(District_URLs.loc[i,'URL'])
    print(District_URLs.loc[i,'District'])
    Tokyo_Dataset = Tokyo_Dataset.append(D)
    pbar.update()
    print(pbar)

    save_path = "E:/UTokyo/Prof_Griffen RA/June_SUUMO/"+str(District_URLs.loc[i,'District']).strip()+"(0611).csv"
    Tokyo_Dataset.to_csv(save_path, encoding = "'utf_8_sig'")
    Tokyo_Dataset = pd.DataFrame()
    time.sleep(10)

0% [#] 100% | ETA: 00:00:00
Total time elapsed: 02:48:11
練馬区
Title: 
  Started: 06/11/2020 23:08:11
  Finished: 06/12/2020 01:56:23
  Total time elapsed: 02:48:11
  CPU %: 13.30
  Memory %: 2.51


23区北部

In [31]:
Tokyo_Dataset = pd.DataFrame()
pbar = pyprind.ProgBar(len(District_URLs[20:23]), monitor=True)
for i in range(20,23):
    D = TurnPage(District_URLs.loc[i,'URL'])
    print(District_URLs.loc[i,'District'])
    Tokyo_Dataset = Tokyo_Dataset.append(D)
    pbar.update()
    print(pbar)

    save_path = "E:/UTokyo/Prof_Griffen RA/June_SUUMO/"+str(District_URLs.loc[i,'District']).strip()+"(0612).csv"
    Tokyo_Dataset.to_csv(save_path, encoding = "'utf_8_sig'")
    Tokyo_Dataset = pd.DataFrame()
    time.sleep(10)

豊島区
Title: 
  Started: 06/12/2020 12:06:29
  Finished: 06/12/2020 14:00:11
  Total time elapsed: 00:00:00
  CPU %: 13.70
  Memory %: 2.41
0% [## ] 100% | ETA: 01:49:19北区
Title: 
  Started: 06/12/2020 12:06:29
  Finished: 06/12/2020 15:45:08
  Total time elapsed: 00:00:00
  CPU %: 14.30
  Memory %: 2.24
0% [###] 100% | ETA: 00:00:00
Total time elapsed: 06:01:19
板橋区
Title: 
  Started: 06/12/2020 12:06:29
  Finished: 06/12/2020 18:07:48
  Total time elapsed: 06:01:19
  CPU %: 14.00
  Memory %: 2.26


都下

In [32]:
Tokyo_Dataset = pd.DataFrame()
pbar = pyprind.ProgBar(len(District_URLs[23:25]), monitor=True)
for i in range(23,25):
    D = TurnPage(District_URLs.loc[i,'URL'])
    print(District_URLs.loc[i,'District'])
    Tokyo_Dataset = Tokyo_Dataset.append(D)
    pbar.update()
    print(pbar)

    save_path = "E:/UTokyo/Prof_Griffen RA/June_SUUMO/"+str(District_URLs.loc[i,'District']).strip()+"(0612).csv"
    Tokyo_Dataset.to_csv(save_path, encoding = "'utf_8_sig'")
    Tokyo_Dataset = pd.DataFrame()
    time.sleep(10)

0% [# ] 100% | ETA: 02:52:21八王子市
Title: 
  Started: 06/12/2020 19:33:35
  Finished: 06/12/2020 22:25:56
  Total time elapsed: 00:00:00
  CPU %: 13.00
  Memory %: 2.45
0% [##] 100% | ETA: 00:00:00
Total time elapsed: 03:50:06
立川市
Title: 
  Started: 06/12/2020 19:33:35
  Finished: 06/12/2020 23:23:41
  Total time elapsed: 03:50:06
  CPU %: 14.00
  Memory %: 2.30


In [33]:
Tokyo_Dataset = pd.DataFrame()
pbar = pyprind.ProgBar(len(District_URLs[25:30]), monitor=True)
for i in range(25,30):
    D = TurnPage(District_URLs.loc[i,'URL'])
    print(District_URLs.loc[i,'District'])
    Tokyo_Dataset = Tokyo_Dataset.append(D)
    pbar.update()
    print(pbar)

    save_path = "E:/UTokyo/Prof_Griffen RA/June_SUUMO/"+str(District_URLs.loc[i,'District']).strip()+"(0613).csv"
    Tokyo_Dataset.to_csv(save_path, encoding = "'utf_8_sig'")
    Tokyo_Dataset = pd.DataFrame()
    time.sleep(10)

0% [#    ] 100% | ETA: 03:41:59武蔵野市
Title: 
  Started: 06/13/2020 01:36:55
  Finished: 06/13/2020 02:32:25
  Total time elapsed: 00:00:00
  CPU %: 12.00
  Memory %: 2.10
0% [##   ] 100% | ETA: 02:52:27三鷹市
Title: 
  Started: 06/13/2020 01:36:55
  Finished: 06/13/2020 03:31:53
  Total time elapsed: 00:00:00
  CPU %: 11.10
  Memory %: 2.12
0% [###  ] 100% | ETA: 01:29:54青梅市
Title: 
  Started: 06/13/2020 01:36:55
  Finished: 06/13/2020 03:51:47
  Total time elapsed: 00:00:00
  CPU %: 10.60
  Memory %: 2.12
0% [#### ] 100% | ETA: 00:51:20府中市
Title: 
  Started: 06/13/2020 01:36:55
  Finished: 06/13/2020 05:02:16
  Total time elapsed: 00:00:00
  CPU %: 11.20
  Memory %: 2.14
0% [#####] 100% | ETA: 00:00:00
Total time elapsed: 03:50:12
昭島市
Title: 
  Started: 06/13/2020 01:36:55
  Finished: 06/13/2020 05:27:07
  Total time elapsed: 03:50:12
  CPU %: 12.10
  Memory %: 2.16


In [ ]:
Tokyo_Dataset = pd.DataFrame()
pbar = pyprind.ProgBar(len(District_URLs[30:31]), monitor=True)
for i in range(30,31):
    D = TurnPage(District_URLs.loc[i,'URL'])
    print(District_URLs.loc[i,'District'])
    Tokyo_Dataset = Tokyo_Dataset.append(D)
    pbar.update()
    print(pbar)

    save_path = "E:/UTokyo/Prof_Griffen RA/June_SUUMO/"+str(District_URLs.loc[i,'District']).strip()+"(0613).csv"
    Tokyo_Dataset.to_csv(save_path, encoding = "'utf_8_sig'")
    Tokyo_Dataset = pd.DataFrame()
    time.sleep(10)

In [36]:
for i in range(31,35):
    print(District_URLs.loc[i,'District'])
    print(District_URLs.loc[i,'URL'])

町田市
https://suumo.jp/jj/chintai/ichiran/FR301FC001/?ar=030&bs=040&ta=13&sc=13209&cb=0.0&ct=9999999&et=9999999&cn=9999999&mb=0&mt=9999999&shkr1=03&shkr2=03&shkr3=03&shkr4=03&fw2=&srch_navi=1
小金井市
https://suumo.jp/jj/chintai/ichiran/FR301FC001/?ar=030&bs=040&ta=13&sc=13210&cb=0.0&ct=9999999&et=9999999&cn=9999999&mb=0&mt=9999999&shkr1=03&shkr2=03&shkr3=03&shkr4=03&fw2=&srch_navi=1
小平市
https://suumo.jp/jj/chintai/ichiran/FR301FC001/?ar=030&bs=040&ta=13&sc=13211&cb=0.0&ct=9999999&et=9999999&cn=9999999&mb=0&mt=9999999&shkr1=03&shkr2=03&shkr3=03&shkr4=03&fw2=&srch_navi=1
日野市
https://suumo.jp/jj/chintai/ichiran/FR301FC001/?ar=030&bs=040&ta=13&sc=13212&cb=0.0&ct=9999999&et=9999999&cn=9999999&mb=0&mt=9999999&shkr1=03&shkr2=03&shkr3=03&shkr4=03&fw2=&srch_navi=1


In [37]:
Tokyo_Dataset = pd.DataFrame()
pbar = pyprind.ProgBar(len(District_URLs[31:35]), monitor=True)
for i in range(31,35):
    D = TurnPage(District_URLs.loc[i,'URL'])
    print(District_URLs.loc[i,'District'])
    Tokyo_Dataset = Tokyo_Dataset.append(D)
    pbar.update()
    print(pbar)

    save_path = "E:/UTokyo/Prof_Griffen RA/June_SUUMO/"+str(District_URLs.loc[i,'District']).strip()+"(0613).csv"
    Tokyo_Dataset.to_csv(save_path, encoding = "'utf_8_sig'")
    Tokyo_Dataset = pd.DataFrame()
    time.sleep(10)

0% [#   ] 100% | ETA: 04:54:29町田市
Title: 
  Started: 06/13/2020 10:38:53
  Finished: 06/13/2020 12:17:03
  Total time elapsed: 00:00:00
  CPU %: 12.50
  Memory %: 2.05
0% [##  ] 100% | ETA: 02:16:57小金井市
Title: 
  Started: 06/13/2020 10:38:53
  Finished: 06/13/2020 12:55:50
  Total time elapsed: 00:00:00
  CPU %: 12.60
  Memory %: 2.05
0% [### ] 100% | ETA: 01:01:23小平市
Title: 
  Started: 06/13/2020 10:38:53
  Finished: 06/13/2020 13:43:03
  Total time elapsed: 00:00:00
  CPU %: 13.00
  Memory %: 2.06
0% [####] 100% | ETA: 00:00:00
Total time elapsed: 04:08:37
日野市
Title: 
  Started: 06/13/2020 10:38:53
  Finished: 06/13/2020 14:47:31
  Total time elapsed: 04:08:37
  CPU %: 13.00
  Memory %: 2.07


In [38]:
Tokyo_Dataset = pd.DataFrame()
pbar = pyprind.ProgBar(len(District_URLs[35:40]), monitor=True)
for i in range(35,40):
    D = TurnPage(District_URLs.loc[i,'URL'])
    print(District_URLs.loc[i,'District'])
    Tokyo_Dataset = Tokyo_Dataset.append(D)
    pbar.update()
    print(pbar)

    save_path = "E:/UTokyo/Prof_Griffen RA/June_SUUMO/"+str(District_URLs.loc[i,'District']).strip()+"(0613).csv"
    Tokyo_Dataset.to_csv(save_path, encoding = "'utf_8_sig'")
    Tokyo_Dataset = pd.DataFrame()
    time.sleep(10)

0% [#    ] 100% | ETA: 02:41:50東村山市
Title: 
  Started: 06/13/2020 16:03:20
  Finished: 06/13/2020 16:43:48
  Total time elapsed: 00:00:00
  CPU %: 28.70
  Memory %: 1.97
0% [##   ] 100% | ETA: 01:55:55国分寺市
Title: 
  Started: 06/13/2020 16:03:20
  Finished: 06/13/2020 17:20:37
  Total time elapsed: 00:00:00
  CPU %: 11.70
  Memory %: 1.96
0% [###  ] 100% | ETA: 01:08:50国立市
Title: 
  Started: 06/13/2020 16:03:20
  Finished: 06/13/2020 17:46:36
  Total time elapsed: 00:00:00
  CPU %: 12.00
  Memory %: 1.95
0% [#### ] 100% | ETA: 00:29:11福生市
Title: 
  Started: 06/13/2020 16:03:20
  Finished: 06/13/2020 18:00:06
  Total time elapsed: 00:00:00
  CPU %: 11.90
  Memory %: 1.95
0% [#####] 100% | ETA: 00:00:00
Total time elapsed: 02:13:06
狛江市
Title: 
  Started: 06/13/2020 16:03:20
  Finished: 06/13/2020 18:16:27
  Total time elapsed: 02:13:06
  CPU %: 11.40
  Memory %: 1.91


In [39]:
Tokyo_Dataset = pd.DataFrame()
pbar = pyprind.ProgBar(len(District_URLs[40:45]), monitor=True)
for i in range(40,45):
    D = TurnPage(District_URLs.loc[i,'URL'])
    print(District_URLs.loc[i,'District'])
    Tokyo_Dataset = Tokyo_Dataset.append(D)
    pbar.update()
    print(pbar)

    save_path = "E:/UTokyo/Prof_Griffen RA/June_SUUMO/"+str(District_URLs.loc[i,'District']).strip()+"(0613).csv"
    Tokyo_Dataset.to_csv(save_path, encoding = "'utf_8_sig'")
    Tokyo_Dataset = pd.DataFrame()
    time.sleep(10)

0% [#    ] 100% | ETA: 01:17:15東大和市
Title: 
  Started: 06/13/2020 18:30:17
  Finished: 06/13/2020 18:49:36
  Total time elapsed: 00:00:00
  CPU %: 10.70
  Memory %: 1.87
0% [##   ] 100% | ETA: 00:52:13清瀬市
Title: 
  Started: 06/13/2020 18:30:17
  Finished: 06/13/2020 19:05:06
  Total time elapsed: 00:00:00
  CPU %: 11.40
  Memory %: 1.86
0% [###  ] 100% | ETA: 00:37:49東久留米市
Title: 
  Started: 06/13/2020 18:30:17
  Finished: 06/13/2020 19:27:02
  Total time elapsed: 00:00:00
  CPU %: 11.30
  Memory %: 1.86
0% [#### ] 100% | ETA: 00:17:37武蔵村山市
Title: 
  Started: 06/13/2020 18:30:17
  Finished: 06/13/2020 19:40:49
  Total time elapsed: 00:00:00
  CPU %: 11.20
  Memory %: 1.87
0% [#####] 100% | ETA: 00:00:00
Total time elapsed: 01:44:45
多摩市
Title: 
  Started: 06/13/2020 18:30:17
  Finished: 06/13/2020 20:15:03
  Total time elapsed: 01:44:45
  CPU %: 11.50
  Memory %: 1.90


In [40]:
Tokyo_Dataset = pd.DataFrame()
pbar = pyprind.ProgBar(len(District_URLs[45:50]), monitor=True)
for i in range(45,50):
    D = TurnPage(District_URLs.loc[i,'URL'])
    print(District_URLs.loc[i,'District'])
    Tokyo_Dataset = Tokyo_Dataset.append(D)
    pbar.update()
    print(pbar)

    save_path = "E:/UTokyo/Prof_Griffen RA/June_SUUMO/"+str(District_URLs.loc[i,'District']).strip()+"(0613).csv"
    Tokyo_Dataset.to_csv(save_path, encoding = "'utf_8_sig'")
    Tokyo_Dataset = pd.DataFrame()
    time.sleep(10)

0% [#    ] 100% | ETA: 01:16:25稲城市
Title: 
  Started: 06/13/2020 22:05:05
  Finished: 06/13/2020 22:24:12
  Total time elapsed: 00:00:00
  CPU %: 10.50
  Memory %: 1.86
0% [##   ] 100% | ETA: 00:49:12羽村市
Title: 
  Started: 06/13/2020 22:05:05
  Finished: 06/13/2020 22:37:53
  Total time elapsed: 00:00:00
  CPU %: 9.70
  Memory %: 1.87
0% [###  ] 100% | ETA: 00:28:56あきる野市
Title: 
  Started: 06/13/2020 22:05:05
  Finished: 06/13/2020 22:48:30
  Total time elapsed: 00:00:00
  CPU %: 9.50
  Memory %: 1.86
0% [#### ] 100% | ETA: 00:21:48西東京市
Title: 
  Started: 06/13/2020 22:05:05
  Finished: 06/13/2020 23:32:18
  Total time elapsed: 00:00:00
  CPU %: 10.90
  Memory %: 1.85
0% [#####] 100% | ETA: 00:00:00
Total time elapsed: 01:33:50
西多摩郡
Title: 
  Started: 06/13/2020 22:05:05
  Finished: 06/13/2020 23:38:56
  Total time elapsed: 01:33:50
  CPU %: 11.80
  Memory %: 1.88


## Check the data 

### Toka index

In [2]:
District_URLs = pd.read_csv("E:/UTokyo/Prof_Griffen RA/District_URLs(0609).csv")
len(District_URLs)

50

In [3]:
adj_index = []
for i in range(23,25):
    a = District_URLs.loc[i,"District"].strip()+"(0612).csv"
    adj_index.append(a)
adj_index

['八王子市(0612).csv', '立川市(0612).csv']

In [4]:
for i in range(25,50):
    a = District_URLs.loc[i,"District"].strip()+"(0613).csv"
    adj_index.append(a)
adj_index

['八王子市(0612).csv',
 '立川市(0612).csv',
 '武蔵野市(0613).csv',
 '三鷹市(0613).csv',
 '青梅市(0613).csv',
 '府中市(0613).csv',
 '昭島市(0613).csv',
 '調布市(0613).csv',
 '町田市(0613).csv',
 '小金井市(0613).csv',
 '小平市(0613).csv',
 '日野市(0613).csv',
 '東村山市(0613).csv',
 '国分寺市(0613).csv',
 '国立市(0613).csv',
 '福生市(0613).csv',
 '狛江市(0613).csv',
 '東大和市(0613).csv',
 '清瀬市(0613).csv',
 '東久留米市(0613).csv',
 '武蔵村山市(0613).csv',
 '多摩市(0613).csv',
 '稲城市(0613).csv',
 '羽村市(0613).csv',
 'あきる野市(0613).csv',
 '西東京市(0613).csv',
 '西多摩郡(0613).csv']

In [5]:
import pandas as pd
import os
Path =  r"E:/UTokyo/Prof_Griffen RA/June_SUUMO"  
file_list = os.listdir(Path) # File name of all the files under "Path"    
SaveFile_Name = 'SUUMO_June_Combined.csv'



In [6]:
Tokyo_toka = pd.DataFrame()
c = 0
for i in adj_index:
    df = pd.read_csv(open("E:/UTokyo/Prof_Griffen RA/June_SUUMO" + "\\" + i, "r", encoding="utf_8_sig"))
    Tokyo_toka = Tokyo_toka.append(df)
    c += 1
    print(c)

八王子市(0612).csv
立川市(0612).csv
武蔵野市(0613).csv
三鷹市(0613).csv
青梅市(0613).csv
府中市(0613).csv
昭島市(0613).csv
調布市(0613).csv
町田市(0613).csv
小金井市(0613).csv
小平市(0613).csv
日野市(0613).csv
東村山市(0613).csv
国分寺市(0613).csv
国立市(0613).csv
福生市(0613).csv
狛江市(0613).csv
東大和市(0613).csv
清瀬市(0613).csv
東久留米市(0613).csv
武蔵村山市(0613).csv
多摩市(0613).csv
稲城市(0613).csv
羽村市(0613).csv
あきる野市(0613).csv
西東京市(0613).csv
西多摩郡(0613).csv


In [9]:
Tokyo_toka.index = range(len(Tokyo_toka))
Tokyo_toka.index

RangeIndex(start=0, stop=69532, step=1)

Number of rows of Tokyo_toka 

In [8]:
Tokyo_toka.shape[0]

69532

In [13]:
Tokyo_toka.loc[1, "District"].split("市",1)[0]+ "市"

'東京都八王子市'

In [14]:
Tokyo_toka.loc[1, "District"]

'東京都八王子市小宮町区'

整理district, 把郡市 以後的去掉

In [15]:
a = b = 0
for i in range(Tokyo_toka.shape[0]):
    if Tokyo_toka.loc[i, "District"].find("郡") == 6:
        Tokyo_toka.loc[i, "District"] = Tokyo_toka.loc[i, "District"].split("郡",1)[0] + "郡"
        a += 1
    else:
        Tokyo_toka.loc[i, "District"] = Tokyo_toka.loc[i, "District"].split("市",1)[0] + "市"
        b += 1

In [17]:
print(a,b, a+b)

421 69111 69532


確定處理的數量沒錯

In [18]:
len(Tokyo_toka["District"].drop_duplicates())

27

不用處理的都內資料叫進來

In [19]:
Tonai_file_list = []
for i in file_list:
    if i in adj_index:
        pass
    else:
        Tonai_file_list.append(i)  # Index of those not in Toka area

In [20]:
Tonai_file_list

['世田谷区(0611).csv',
 '中央区(0609).csv',
 '中野区(0509).csv',
 '北区(0612).csv',
 '千代田区(0609).csv',
 '台東区(0610).csv',
 '品川区(0611).csv',
 '墨田区(0610).csv',
 '大田区(0611).csv',
 '文京区(0609).csv',
 '新宿区(0609).csv',
 '杉並区(0509).csv',
 '板橋区(0612).csv',
 '江戸川区(0610).csv',
 '江東区(0610).csv',
 '渋谷区(0609).csv',
 '港区(0609).csv',
 '目黒区(0611).csv',
 '練馬区(0611).csv',
 '荒川区(0610).csv',
 '葛飾区(0610).csv',
 '豊島区(0612).csv',
 '足立区(0610).csv']

In [21]:
len(Tonai_file_list)

23

In [31]:
# ddf = pd.read_csv(open("E:/UTokyo/Prof_Griffen RA/May_SUUMO" + "\\" + file_list[0], "r",encoding="utf_8_sig"))
# ddf.to_csv(Path + '\\' + SaveFile_Name, index=False)

Export Toka Dataset

In [22]:
Tokyo_toka.to_csv(Path + '\\' + SaveFile_Name, index=False)

In [23]:
l = len(Tokyo_toka)
for i in range(0, len(Tonai_file_list)):
    ddf = pd.read_csv(open("E:/UTokyo/Prof_Griffen RA/June_SUUMO" + "\\" + Tonai_file_list[i], "r",encoding="utf_8_sig"))
    ddf.to_csv(Path+'\\'+ SaveFile_Name,encoding="utf_8_sig",index=False, header=False, mode='a+') #attach new data to Toka dataset
    l = l + len(ddf)
    print(l)

85281
89994
97694
104151
106592
112475
120812
126161
137129
141778
152909
164753
173654
182127
189766
197344
205437
210886
222038
224783
229914
237173
245257


In [24]:
SUUMO_June = pd.read_csv(Path + "\\" + SaveFile_Name)

In [25]:
# Wierd. Unclear Purpose.
Sampling = pd.DataFrame()
for i in range(0, len(file_list)):
    ddf = pd.read_csv(open("E:/UTokyo/Prof_Griffen RA/June_SUUMO" + "\\" + file_list[i], "r",encoding="utf_8_sig"))
    Sampling = Sampling.append(ddf.sample(1))

In [26]:
len(SUUMO_June)

245257

In [27]:
D = SUUMO_June["District"].drop_duplicates()
len(D)   

50

In [29]:
Samples = pd.DataFrame()

for i in D:
     n = SUUMO_June[SUUMO_June["District"] == i]
     n = n.sample(n = 50)
     Samples = Samples.append(n)


In [30]:
Samples.to_csv(Path + "\\" + "Samples.csv",encoding="utf_8_sig",index=False)

In [31]:
DD = Samples["District"].drop_duplicates()  

In [32]:
len(DD)

50